# PART 1: RETRIEVING INFORMATION
## Excercise 1

In [ ]:
SELECT unitprice, quantityperunit
FROM northwind.products
WHERE productname = 'Geitost'

## Excercise 2

In [ ]:
SELECT productname, unitsinstock
FROM northwind.products
WHERE quantityperunit LIKE '%jars' OR productname LIKE '%Sauce'

## Excercise 3

In [ ]:
SELECT companyname
FROM northwind.suppliers
WHERE country = 'USA' AND (fax IS NOT NULL OR homepage IS NOT NULL)

## Excercise 4

In [ ]:
SELECT DISTINCT firstname, lastname
FROM northwind.employees
INNER JOIN northwind.orders
USING (employeeid) 
WHERE freight > 200

## Excercise 5

In [ ]:
SELECT companyname, address, city
FROM northwind.customers
WHERE country = (SELECT country
    FROM northwind.customers
    WHERE companyname = 'Eastern Connection')

## Excercise 6

In [ ]:
SELECT productname, unitprice
FROM northwind.products
INNER JOIN northwind.categories
USING(categoryid)
WHERE categoryname = 'Beverages' AND unitprice > 20

## Excercise 7

In [ ]:
SELECT COUNT(*) AS Nbr_of_orders_1996
FROM northwind.orders
WHERE EXTRACT(YEAR from orderdate) = 1996

## Excercise 8

In [ ]:
SELECT categoryname, COUNT(productid) AS Nbr_of_products
FROM northwind.categories
INNER JOIN northwind.products
USING(categoryid)
GROUP BY categoryname
ORDER BY Nbr_of_products DESC

## Excercise 9

In [ ]:
SELECT AVG(o) AS avg_products_per_order 
FROM (
    SELECT COUNT(productid) AS o
    FROM northwind.order_details
    GROUP BY orderid
) a

## Excercise 10

In [ ]:
SELECT e.firstname, e.lastname
FROM northwind.employees AS e,
    (SELECT o1.employeeid, SUM(ov.orders_value) AS tot_order_value
    FROM northwind.orders AS o1,
        (SELECT o2.orderid, SUM(o2.unitprice*o2.quantity *(1-o2.discount)) AS orders_value
        FROM northwind.order_details AS o2
        GROUP BY o2.orderid) AS ov
    WHERE o1.orderid = ov.orderid
    GROUP BY employeeID) AS op
WHERE op.employeeid = e.employeeid AND tot_order_value > 200000
ORDER BY e.birthdate DESC
LIMIT 1


## Excercise 11

In [ ]:
SELECT hfirst.hiredfirst, hlast.hiredlast
FROM (SELECT e1.firstname || ' ' || e1.lastname AS hiredfirst
    FROM northwind.employees AS e1
    ORDER BY e1.hiredate
     LIMIT 1) AS hfirst,
    (SELECT e2.firstname || ' ' || e2.lastname AS hiredlast
    FROM northwind.employees AS e2
    ORDER BY e2.hiredate DESC
    LIMIT 1) AS hlast


# PART 2: DATABASE DESIGN AND CREATION
## Excercise 1

In [ ]:
Movies:
- MID (int), primary key
- moviename (text)

Actors:
- AID (int), primary key
- actorname (text)
- actorbirthdate (date)

Cinemas:
- CID (int), primary key
- cinemaname (text)

CinemaShowtimes:
- SID (int), primary key
- MID (int), references Movies' MID column
- CID (int), references Cinemas' CID column
- cdate (date)

MovieActors:
- MID (int), references Movies' MID column
- AID (int), references Actors' AID column


## Excercise 2

In [ ]:
CREATE TABLE Movies (
    MID SERIAL PRIMARY KEY,
    moviename text NOT NULL);

CREATE TABLE Actors (
    AID SERIAL PRIMARY KEY,
    actorname text NOT NULL,
    actorbirthdate date);

CREATE TABLE Cinemas (
    CID SERIAL PRIMARY KEY,
    cinemaname text NOT NULL);

CREATE TABLE CinemaShowtimes (
    SID SERIAL PRIMARY KEY,
    MID int REFERENCES Movies (MID),
    CID int REFERENCES Cinemas (CID),
    cdate date);

CREATE TABLE MovieActors (
    MID int REFERENCES Movies (MID),
    AID int REFERENCES Actors (AID));


## Excercise 3

In [ ]:
CREATE TABLE MovieActorCinema (
    moviename text,
    actor text,
    actorbirthdate date,
    cinema text,
    played date);

INSERT INTO MovieActorCinema
VALUES  ('Jurassic Park', 'Sam Neill', '1947-09-14', 'Ringen Cinema', '1993-07-01'),
        ('Jurassic Park', 'Sam Neill', '1947-09-14', 'Colloseum', '1993-07-03'),
        ('Free Willy', 'Lori Petty', '1963-10-14', 'Ringen Cinema', '1993-07-13'),
        ('Free Willy', 'Jason Richter', '1980-01-29', 'Ringen Cinema', '1993-07-13');

INSERT INTO Movies (moviename)
SELECT DISTINCT(moviename)
FROM MovieActorCinema;

INSERT INTO Actors (actorname, actorbirthdate)
SELECT DISTINCT(actor), actorbirthdate
FROM MovieActorCinema;

INSERT INTO Cinemas (cinemaname)
SELECT DISTINCT(cinema)
FROM MovieActorCinema;

INSERT INTO movieactors(mid, aid)
SELECT DISTINCT(m.mid), a.aid
FROM movieactorcinema AS mac,
    (SELECT mid, moviename
    FROM movies) AS m,
    (SELECT aid, actorname
    FROM actors) AS a
WHERE mac.moviename = m.moviename AND mac.actor = a.actorname
ORDER BY aid;

INSERT INTO cinemashowtimes(mid,cid,cdate)
SELECT DISTINCT(m.mid), c.cid, mac.played
FROM movieactorcinema AS mac,
    (SELECT mid, moviename
    FROM movies) AS m,
    (SELECT cid, cinemaname
    FROM cinemas) AS c
WHERE mac.moviename = m.moviename AND mac.cinema = c.cinemaname
ORDER BY mid;



# PART 3: PYTHON AND SQL
## Excercise 1

In [ ]:
import psycopg2
import matplotlib.pyplot as plt

connection = "dbname='s1316704DB' port='5432' user='s1316704' " + \
             "host='postgresql-lb-1715985356.eu-west-1.elb.amazonaws.com' " + \
             "password='UIOoju559'"

def analysis():
    conn = psycopg2.connect(connection)

    ch = 0
    while (ch != 3):
        print(" -- ANALYZER --")
        print("Please choose an option:")
        print("1. Today's stats")
        print("2. Plot orders")
        print("3. Exit")
        ch = int(input("Option: "))

        if (ch == 1):
            todays_stats(conn)
        elif (ch == 2):
            plot_orders(conn)


def todays_stats(conn):

    cur = conn.cursor()

    q = "SELECT sum(p.price * o.num) FROM products as p, orders as o WHERE p.pid = o.pid and o.date = current_date"
    cur.execute(q)
    totaltoday = cur.fetchall()

    r = "select p.name FROM orders as o, products as p WHERE o.pid = p.pid and o.date = current_date ORDER BY o.num DESC LIMIT 1"
    cur.execute(r)
    bestseller = cur.fetchall()

    for i in totaltoday:
        print('\nThe store has sold a total of %s today.' % str(i[0]))
    for p in bestseller:
        print('The most popular product today is %s.\n' % str(p[0]))


analysis()

## Excercise 2

In [ ]:
import psycopg2
import matplotlib.pyplot as plt

connection = "dbname='s1316704DB' port='5432' user='s1316704' " + \
             "host='postgresql-lb-1715985356.eu-west-1.elb.amazonaws.com' " + \
             "password='UIOoju559'"

def analysis():
    conn = psycopg2.connect(connection)

    ch = 0
    while (ch != 3):
        print(" -- ANALYZER --")
        print("Please choose an option:")
        print("1. Today's stats")
        print("2. Plot orders")
        print("3. Exit")
        ch = int(input("Option: "))

        if (ch == 1):
            todays_stats(conn)
        elif (ch == 2):
            plot_orders(conn)


def todays_stats(conn):

    cur = conn.cursor()

    q = "SELECT sum(p.price * o.num) FROM products as p, orders as o WHERE p.pid = o.pid and o.date = current_date"
    cur.execute(q)
    totaltoday = cur.fetchall()

    r = "select p.name FROM orders as o, products as p WHERE o.pid = p.pid and o.date = current_date ORDER BY o.num DESC LIMIT 1"
    cur.execute(r)
    bestseller = cur.fetchall()

    for i in totaltoday:
        print('\nThe store has sold a total of %s today.' % str(i[0]))
    for p in bestseller:
        print('The most popular product today is %s.\n' % str(p[0]))


def plot_orders(conn):

    cur = conn.cursor()
    product_search = input('Enter product name: ')

    if product_search == '':
        s = "SELECT o.date, SUM(p.price * o.num) FROM products as p, orders as o WHERE p.pid = o.pid GROUP BY o.date ORDER BY o.date"
        cur.execute(s)
        all_orders = cur.fetchall()

        date = [0,]
        sales = []
        for something in all_orders:
            date.append(date[-1]+1)
            sales.append(something[1])
        date.pop()

        plt.style.use('seaborn')
        plt.plot(date, sales)
        plt.title('Total sold per day for all products')
        plt.xlabel("Day")
        plt.ylabel("Sales")
        plt.grid(True)
        plt.show()


    else:
        s = "SELECT o.date, SUM(p.price * o.num), p.name FROM products as p, orders as o WHERE p.pid = o.pid and p.name = '%s' GROUP BY o.date, p.name ORDER BY o.date" % (product_search)
        cur.execute(s)
        all_orders = cur.fetchall()

        date = [0,]
        sales = []
        for something in all_orders:
            date.append(date[-1]+1)
            sales.append(something[1])
        date.pop()

        plt.style.use('seaborn')
        plt.plot(date, sales)
        plt.title('Total sold per day for %s' % product_search)
        plt.xlabel("Day")
        plt.ylabel("Sales")
        plt.grid(True)
        plt.show()

analysis()